In [1]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from datetime import datetime

In [2]:
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

load voter data

In [3]:
voter = pd.read_csv('20220512_voter_race_location.csv', low_memory = False)

In [4]:
voter1 = voter[pd.isna(voter.lat) == False].copy()

In [5]:
len(voter1)/len(voter)

0.99778183194305

In [6]:
addresses = voter1.iloc[:,[4,5,6,7,8,9]].copy().drop_duplicates()
addressesgdf = gpd.GeoDataFrame(addresses, geometry=gpd.points_from_xy(addresses['long'], addresses['lat']))
addressesgdf.crs = 'EPSG:4326'

In [7]:
del voter
del voter1

load isochrones

In [8]:
drivingchrones = gpd.read_file('20220521_driving_isochrone_10_15_20_30.shp')
transitchrones = gpd.read_file('20220523_transit_isochrones_10_15_20_30.shp')
drivingchrones.lat = np.round(drivingchrones.lat,6)
drivingchrones.long = np.round(drivingchrones.long,6)
transitchrones.lat = transitchrones.lat.astype('float')
transitchrones.long = transitchrones.long.astype('float')

In [9]:
inyear = transitchrones[['in_2022','in_2021','in_nov_202','lat','long']].copy().drop_duplicates()

In [13]:
drivingchrones.head()

,travel_tim,lat,long,GEOID,in_2022,in_nov_202,geometry
0,600.0,31.885024,-85.100646,13239,1,1,"POLYGON ((-85.12078 31.89331, -85.12029 31.893..."
1,1200.0,31.885024,-85.100646,13239,1,1,"MULTIPOLYGON (((-85.11114 31.89879, -85.11108 ..."
2,1800.0,31.885024,-85.100646,13239,1,1,"MULTIPOLYGON (((-85.12649 31.78024, -85.12894 ..."
3,900.0,31.885024,-85.100646,13239,1,1,"MULTIPOLYGON (((-85.12078 31.89331, -85.12029 ..."
4,600.0,31.293249,-82.842304,13003,1,1,"POLYGON ((-82.97390 31.32202, -82.96326 31.322..."


In [14]:
drivingvoterjoin = gpd.sjoin(addressesgdf,drivingchrones, how = 'left')

In [15]:
drivingvoterjoin1 = drivingvoterjoin.drop('geometry', axis = 1)

In [16]:
drivingvoterjoin2 =drivingvoterjoin1.drop_duplicates()

In [17]:
drivingvoterjoin2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56469070 entries, 0 to 7552177
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   lat_left         float64
 5   long_left        float64
 6   index_right      float64
 7   travel_tim       float64
 8   lat_right        float64
 9   long_right       float64
 10  GEOID            object 
 11  in_2022          object 
 12  in_nov_202       object 
dtypes: float64(6), object(7)
memory usage: 5.9+ GB


In [18]:
drivingvoterjoin2.to_csv('20220521_voter_address_driving_isochrone_join.csv')

In [19]:
del drivingchrones
del drivingchrones1

In [9]:
transitjoin = gpd.sjoin(addressesgdf, transitchrones, how = 'left')

In [10]:
transitjoin1 = transitjoin.drop('geometry', axis = 1)

In [11]:
transitjoin1.to_csv('20220523_voter_addr_transitchrone_join.csv')

joining addresses with geographies

In [21]:
urban = gpd.read_file('tl_2021_us_uac10/tl_2021_us_uac10.shp')
GA = urban[urban.NAME10.str.contains('GA')].copy()

In [38]:
GAurban = GA[['geometry']].copy().reset_index()

In [39]:
GAurban['urban'] = True

In [41]:
GAurban=GAurban.to_crs('EPSG:4326')

In [74]:
GAurban1 = GAurban.drop('index', axis = 1)

In [84]:
GAurban1.head()

,geometry,urban
0,"POLYGON ((-82.71957 33.72904, -82.71805 33.728...",True
1,"MULTIPOLYGON (((-82.60097 31.84351, -82.60098 ...",True
2,"POLYGON ((-81.91131 32.15790, -81.91141 32.156...",True
3,"MULTIPOLYGON (((-83.44627 31.11137, -83.44635 ...",True
4,"MULTIPOLYGON (((-82.33667 32.20602, -82.33522 ...",True


In [77]:
# referencing urban area
addrjoin1 = gpd.sjoin(addressesgdf, GAurban1, how = 'left')

In [82]:
addrjoin1.urban.value_counts()

True    2269538
Name: urban, dtype: int64

In [83]:
2269538/len(addrjoin1)

0.7109161627897143

In [78]:
tracts = gpd.read_file('tl_2021_13_tract/tl_2021_13_tract.shp')

In [93]:
tracts = tracts.to_crs('EPSG:4326')

In [90]:
addrjoin1 = addrjoin1.drop('index_right', axis = 1)

In [94]:
addrjoin2 = gpd.sjoin(addrjoin1, tracts, how = 'left')

In [97]:
addrjoin2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3192437 entries, 0 to 7552177
Data columns (total 21 columns):
 #   Column           Dtype   
---  ------           -----   
 0   st_address       object  
 1   RESIDENCE_CITY   object  
 2   RESIDENCE_STATE  object  
 3   ZIP5             object  
 4   lat_left         float64 
 5   long_left        float64 
 6   geometry         geometry
 7   urban            object  
 8   index_right      float64 
 9   STATEFP          object  
 10  COUNTYFP         object  
 11  TRACTCE          object  
 12  GEOID            object  
 13  NAME             object  
 14  NAMELSAD         object  
 15  MTFCC            object  
 16  FUNCSTAT         object  
 17  ALAND            float64 
 18  AWATER           float64 
 19  INTPTLAT         object  
 20  INTPTLON         object  
dtypes: float64(5), geometry(1), object(15)
memory usage: 535.8+ MB


In [98]:
addrjoin2.drop('geometry', axis = 1).to_csv('20220515_voter_address_tract_ua.csv')

do the same spatial joins for public transit isochrones 